In [1]:
import pydogpack.utils.quadrature as quadrature
import numpy as np

# Quadrature

## 1D quadrature

### Quadrature Points and Weights

In [3]:
quad_order = 5
tuple_ = quadrature.gauss_pts_and_wgts_1d_canonical(quad_order)
quad_pts = tuple_[0]
quad_wgts = tuple_[1]
print("Quadrature Points")
print(quad_pts)
print("Quadrature Weights")
print(quad_wgts)

Quadrature Points
[-0.90617985 -0.53846931  0.          0.53846931  0.90617985]
Quadrature Weights
[0.23692689 0.47862867 0.56888889 0.47862867 0.23692689]


### Quadrature Example

In [33]:
quad_order = 5
f = lambda x: np.array([np.cos(x), 2.0 * np.cos(x)])
F = lambda x: np.array([np.sin(x), 2.0 * np.sin(x)])
quad_integral = quadrature.gauss_quadrature_1d(f, 1.0, 2.0, quad_order)
exact_integral = F(2.0) - F(1.0)
error = exact_integral - quad_integral
print("Quadrature Result")
print(quad_integral)
print("Exact Integral")
print(exact_integral)
print("Error")
print(error)

Quadrature Result
[0.06782644 0.13565288]
Exact Integral
[0.06782644 0.13565288]
Error
[-2.77555756e-14 -5.55111512e-14]


In [34]:
f

<function __main__.<lambda>(x)>

### Test Quadrature Exactness

Gauss Legendre Quadrature of order n is exact for polynomials of degree 2n - 1 or less

In [14]:
max_quad_order = 5
tolerance = 1e-15
for quad_order in range(1, max_quad_order + 1):
    for degree in range(2 * quad_order):
        f = lambda x: np.power(x, degree)
        F = lambda x: np.power(x, degree + 1) / (degree + 1)
        quad_integral = quadrature.gauss_quadrature_1d_canonical(f, quad_order)
        exact_integral = F(1) - F(-1)
        error = abs(exact_integral - quad_integral)
        assert error <= tolerance

## 2D Quadrature on Rectangles

### Quadrature Points and weights

In [28]:
quad_order = 3
tuple_ = quadrature.gauss_pts_and_wgts_2d_canonical(quad_order)
quad_pts = tuple_[0]
quad_wgts = tuple_[1]
print("Quadrature Points")
print(quad_pts)
print("Quadrature Weights")
print(quad_wgts)

Quadrature Points
[[-0.77459667 -0.77459667]
 [-0.77459667  0.        ]
 [-0.77459667  0.77459667]
 [ 0.         -0.77459667]
 [ 0.          0.        ]
 [ 0.          0.77459667]
 [ 0.77459667 -0.77459667]
 [ 0.77459667  0.        ]
 [ 0.77459667  0.77459667]]
Quadrature Weights
[0.30864198 0.49382716 0.30864198 0.49382716 0.79012346 0.49382716
 0.30864198 0.49382716 0.30864198]


### Quadrature Example

In [53]:
quad_order = 5
x_left = 0.0
x_right = 1.0
y_bottom = 0.0
y_top = 2.0
f = lambda x: np.array([np.cos(x[..., 0]), 2.0 * np.exp(x[..., 1])])
F = lambda x: np.array([np.sin(x[..., 0]) * x[..., 1], 2.0 * np.exp(x[..., 1]) * x[..., 0]])
quad_integral = quadrature.gauss_quadrature_2d(f, x_left, x_right, y_bottom, y_top, quad_order)
exact_integral = (F(np.array([x_right, y_top])) 
    - F(np.array([x_left, y_top])) 
    - F(np.array([x_right, y_bottom])) 
    + F(np.array([x_left, y_bottom])))
error  = abs(exact_integral - quad_integral)
print("Quadrature Result")
print(quad_integral)
print("Exact Integral")
print(exact_integral)
print("Error")
print(error)

Quadrature Result
[ 1.68294197 12.77811219]
Exact Integral
[ 1.68294197 12.7781122 ]
Error
[6.89226454e-13 4.48394921e-09]


In [51]:
x = quad_pts
np.sin(x[..., 0]) * x[..., 1]

array([ 0.5417745, -0.       , -0.5417745, -0.       ,  0.       ,
        0.       , -0.5417745,  0.       ,  0.5417745])

In [24]:
quad_pts_1d = tuple_[0]
quad_wgts_1d = tuple_[1]
quad_pts_2d = np.array([[quad_pts_1d[i], quad_pts_1d[j]] for i in range(quad_order) for j in range(quad_order)])
quad_wgts_2d = np.array([quad_wgts_1d[i] * quad_wgts_1d[j] for i in range(quad_order) for j in range(quad_order)])

In [27]:
quad_pts_2d.shape

(25, 2)

In [25]:
quad_wgts_2d

array([0.05613435, 0.1134    , 0.13478507, 0.1134    , 0.05613435,
       0.1134    , 0.2290854 , 0.27228653, 0.2290854 , 0.1134    ,
       0.13478507, 0.27228653, 0.32363457, 0.27228653, 0.13478507,
       0.1134    , 0.2290854 , 0.27228653, 0.2290854 , 0.1134    ,
       0.05613435, 0.1134    , 0.13478507, 0.1134    , 0.05613435])

In [19]:
np.kron(quad_pts_1d, quad_pts_1d)

array([ 0.82116191,  0.48795004, -0.        , -0.48795004, -0.82116191,
        0.48795004,  0.2899492 , -0.        , -0.2899492 , -0.48795004,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
       -0.48795004, -0.2899492 ,  0.        ,  0.2899492 ,  0.48795004,
       -0.82116191, -0.48795004,  0.        ,  0.48795004,  0.82116191])